In [1]:
import pandas as pd
import spacy
from spacy.lang.ru.examples import sentences 
from nltk.stem.snowball import SnowballStemmer
nlp = spacy.load("ru_core_news_sm")
import re

In [2]:
comments = pd.read_csv('comments.csv')[:5000]
comments['text'] = comments.text.astype(str)
comments['text'] = comments.text.apply(lambda x: ' '.join(x.split()[:64]))

### comments are very dirty so we clean them for rfidf use lemmatization and stamming

In [3]:
def clean_comment(text):
    regex = re.compile('[^а-яА-Я]')
    text = ' '.join([i for i in regex.sub(' ', text.lower()).split()])
    doc = nlp(text)
    text = ' '.join([token.lemma_ for token in doc])
    return text
comments['text_lemm'] = comments.text.apply(lambda text: clean_comment(text))

In [4]:
def clean_comment(text):
    regex = re.compile('[^а-яА-Я]')
    stemmer = SnowballStemmer("russian")
    text = ' '.join([stemmer.stem(i) for i in regex.sub(' ', text.lower()).split()])
    return text
comments['text_stem'] = comments.text.apply(lambda text: clean_comment(text))

In [5]:
comments.head()

,text,likes,text_lemm,text_stem
0,"А сейчас, свежая рубрика: ""Коронановости""",8.0,а сейчас свежий рубрика коронановости,а сейчас свеж рубрик коронанов
1,А почему статистика начала рости ? Вакцина пер...,12.0,а почему статистика начать рости вакцина перес...,а поч статистик нача рост вакцин переста помога
2,"И всё не те мрут, кто надо.",6.0,и вс не тот мереть кто надо,и вс не те мрут кто над
3,Леонида Куравлёва госпитализировали 5 января в...,1.0,леонид куравл ва госпитализировать январь в ви...,леонид куравл ва госпитализирова январ в вирус...
4,"Так это всё смешно, то что в России всё по сце...",37.0,так это вс смешной то что в россия вс по сцена...,так эт вс смешн то что в росс вс по сценар сей...


In [6]:
comments = comments[(comments.text_stem != '') & (comments.text_lemm.notna())]
comments

,text,likes,text_lemm,text_stem
0,"А сейчас, свежая рубрика: ""Коронановости""",8.0,а сейчас свежий рубрика коронановости,а сейчас свеж рубрик коронанов
1,А почему статистика начала рости ? Вакцина пер...,12.0,а почему статистика начать рости вакцина перес...,а поч статистик нача рост вакцин переста помога
2,"И всё не те мрут, кто надо.",6.0,и вс не тот мереть кто надо,и вс не те мрут кто над
3,Леонида Куравлёва госпитализировали 5 января в...,1.0,леонид куравл ва госпитализировать январь в ви...,леонид куравл ва госпитализирова январ в вирус...
4,"Так это всё смешно, то что в России всё по сце...",37.0,так это вс смешной то что в россия вс по сцена...,так эт вс смешн то что в росс вс по сценар сей...
...,...,...,...,...
2,Ах вот как это называется… 🗿,13.0,ах вот как это называться,ах вот как эт называ
3,Кто нанял послелователя Оурелла? Пять,195.0,кто нанять послелователя оурелла пять,кто наня послеловател оурелл пят
5,И в чем он не прав?,6.0,и в что он не правый,и в чем он не прав
6,"Интересно, когда его начнут в школах преподавать",3.0,интересный когда его начать в школа преподавать,интересн когд ег начнут в школ преподава


### reduse number of words in vocabulary

In [7]:
from collections import Counter
import numpy as np
vocab = Counter(np.sum(comments.text_stem.apply(lambda x: x + ' ').values).split())
print('before:', len(vocab))

comments['text_stem'] = comments.text_stem.apply(lambda x: ' '.join([i if vocab[i] > 2 else '[UNK]' for i in x.split()]))

vocab = Counter(np.sum(comments.text_stem.apply(lambda x: x + ' ').values).split())
print('after:', len(vocab))

before: 10665
after: 3404


In [8]:
vocab = Counter(np.sum(comments.text_lemm.apply(lambda x: x + ' ').values).split())
print(len(vocab))

comments['text_lemm'] = comments.text_lemm.apply(lambda x: ' '.join([i if vocab[i] > 2 else '[UNK]' for i in x.split()]))

vocab = Counter(np.sum(comments.text_lemm.apply(lambda x: x + ' ').values).split())
print(len(vocab))

12211
3361


In [9]:
comments.to_csv('comments_lemm_stemm.csv', index_label=False)